In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score
import torch.nn as nn
import pandas as pd

from src.data import parallel_line, orthogonal, triangle, lines_3D, real_data_loader, add_noise_data
import math, csv
import os.path

In [3]:
from src.models import Autoencoder, CNN_Autoencoder
from src.train import train_AE, train_TAE

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## Clustering

### synthetic data

In [6]:
# kmeans_ari, ae_ari, tae_ari gives the ari of respective algorithms
# hyperparameters: repetition, epochs, lr, reg,  
repetition = 5

np.random.seed(24)
results_dict = dict()

csv_file = "results/ae_clust_synthetic.csv"
if not os.path.isfile(csv_file):
    fields = ["data", "rep", "X", "Y", "shape", "noise",
            "kmeans_labels", "kmeans_ari",
            "ae1_num_params", "ae1_lr", "ae1_epochs", "ae1_final_train_loss", "ae1_labels", "ae1_ari", 
            "ae2_num_params", "ae2_lr", "ae2_epochs", "ae2_final_train_loss", "ae2_labels", "ae2_ari", 
            "ae3_num_params", "ae3_lr", "ae3_epochs", "ae3_final_train_loss", "ae3_labels", "ae3_ari",
            "tae1_num_params", "tae1_lr", "tae1_epochs", "tae1_final_train_loss", "tae1_labels", "tae1_ari",
            "tae2_num_params", "tae2_lr", "tae2_epochs", "tae2_final_train_loss", "tae2_labels", "tae2_ari",
            "ptae_num_params", "ptae_lr", "ptae_epochs", "ptae_final_train_loss", "ptae_labels", "ptae_ari", 
            ]
    with open(csv_file, "a") as f:
        writer = csv.writer(f)
        writer.writerow(fields)

lr_l = [0.001, 0.01, 0.1]
epochs = 150
for (idx, dataset, data_generation) in [
                            (0, 'parallel_line', parallel_line(noise=0)), 
                            (1, 'orthogonal', orthogonal(noise=0)), 
                            (2, 'triangle', triangle(noise=0)), 
                            (3, 'lines_3D', lines_3D(noise=0))
                             ]:
    print("shape of dataset: ", data_generation[0].shape)
    for rep in range(repetition):
        print(rep)
        X,Y,X_noise,n_clusters = data_generation
        x_idx = torch.tensor(np.arange(X.shape[0]))
        shuffle_idx = torch.randperm(x_idx.shape[0])
        x_idx = x_idx[shuffle_idx]
        X = X[x_idx].float().to(device)
        Y = Y[x_idx].float().to(device)
        
        # k means
        k = KMeans(n_clusters=n_clusters, random_state=rep, n_init=10).fit(X.cpu())
        k_ari = adjusted_rand_score(k.labels_, Y.cpu())
        print('kmeans ', k_ari)
         
        for lr in lr_l:
            res = [dataset, rep, X.cpu().detach().numpy().tolist(), Y.cpu().detach().numpy().tolist(), list(data_generation[0].shape), 0,
                  k.labels_.tolist(), k_ari]
            
            # Standard one layer AE1 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[1]).to(device)
            ae_l = train_AE(net,X.clone(),Y.clone(), lr=lr,epochs=epochs)
            X_embed = net.get_embedding(X).cpu().detach().numpy()
            kmeans = KMeans(n_clusters=n_clusters, random_state=0,n_init=10).fit(X_embed)
            AE_clust_assign = kmeans.labels_
            ari = adjusted_rand_score(AE_clust_assign, Y.cpu())
            ae1_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae1_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])
            res.append(AE_clust_assign.tolist())
            res.append(ari)
            print('AE1 ', ari)

            # Standard two layer AE2 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[2,1]).to(device)
            ae_l = train_AE(net,X.clone(),Y.clone(), lr=lr,epochs=epochs)
            X_embed = net.get_embedding(X).cpu().detach().numpy()
            kmeans = KMeans(n_clusters=n_clusters, random_state=0,n_init=10).fit(X_embed)
            AE_clust_assign = kmeans.labels_
            ari = adjusted_rand_score(AE_clust_assign, Y.cpu())
            ae2_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae2_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])
            res.append(AE_clust_assign.tolist())
            res.append(ari)
            print('AE2 ', ari)

            # Standard two layer AE3 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[2,1*n_clusters]).to(device)
            ae_l = train_AE(net,X.clone(),Y.clone(), lr=lr,epochs=epochs)
            X_embed = net.get_embedding(X).cpu().detach().numpy()
            kmeans = KMeans(n_clusters=n_clusters, random_state=0,n_init=10).fit(X_embed)
            AE_clust_assign = kmeans.labels_
            ari = adjusted_rand_score(AE_clust_assign, Y.cpu())
            ae3_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae3_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])
            res.append(AE_clust_assign.tolist())
            res.append(ari)
            print('AE3 ', ari)

            # Tensorized AE1 - one layer
            net, tae_l, TAE_clust_assign, _, _, _ = train_TAE(X.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, embed_l=[1], shared=False, epochs=epochs)
            TAE_clust_assign = torch.argmax(TAE_clust_assign,axis=0).cpu().detach().numpy()
            ari = adjusted_rand_score(TAE_clust_assign, Y.cpu())
            tae1_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(tae1_params)
            res.append(lr)
            res.append(epochs)
            res.append(tae_l[-1])
            res.append(TAE_clust_assign.tolist())
            res.append(ari)
            print('TAE1 ', ari)

            # Tensorized AE2 - two layer
            net, tae_l, TAE_clust_assign, _, _, _ = train_TAE(X.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, embed_l=[2,1], shared=False, epochs=epochs)
            TAE_clust_assign = torch.argmax(TAE_clust_assign,axis=0).cpu().detach().numpy()
            ari = adjusted_rand_score(TAE_clust_assign, Y.cpu())
            tae2_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(tae2_params)
            res.append(lr)
            res.append(epochs)
            res.append(tae_l[-1])
            res.append(TAE_clust_assign.tolist())
            res.append(ari)
            print('TAE2 ', ari)

            # Shared TAE
            net, shared_l, SHARED_clust_assign, _, _, _ = train_TAE(X.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, epochs=epochs, embed_l=[2,1], shared=True)
            PTAE_clust_assign = torch.argmax(SHARED_clust_assign,axis=0).cpu().detach().numpy()
            ari = adjusted_rand_score(PTAE_clust_assign, Y.cpu())
            ptae_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ptae_params)
            res.append(lr)
            res.append(epochs)
            res.append(shared_l[-1])
            res.append(PTAE_clust_assign.tolist())
            res.append(ari)
            print('PTAE ', ari)

            with open(csv_file, "a") as f:
                writer = csv.writer(f)
                writer.writerow(res)

shape of dataset:  torch.Size([300, 6])
3
kmeans  0.6488571581544602
Number of parameters in AE:  12
AE1  0.5404621785177707
Number of parameters in AE:  28
AE2  0.5404621785177707
Number of parameters in AE:  36
AE3  0.7525664019930466
Number of parameters in TAE:  36
epoch 149 loss 0.47929306640289726
TAE1  0.5931258064824382
Number of parameters in TAE:  84
epoch 149 loss 0.33466492355335503
TAE2  0.5459658056596445
Number of parameters in TAE:  36
epoch 149 loss 0.3701238912646659
PTAE  0.6683018797932874
Number of parameters in AE:  12
AE1  0.5404621785177707
Number of parameters in AE:  28
AE2  0.5404621785177707
Number of parameters in AE:  36
AE3  0.5404621785177707
Number of parameters in TAE:  36
epoch 149 loss 0.08245556292124093
TAE1  1.0
Number of parameters in TAE:  84
epoch 149 loss 0.027487855409660065
TAE2  1.0
Number of parameters in TAE:  36
epoch 149 loss 0.1907978732076784
PTAE  1.0
Number of parameters in AE:  12
AE1  0.49200971858993836
Number of parameters in AE

/tmp/ipykernel_1061906/351470483.py:68: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0,n_init=10).fit(X_embed)


AE2  0.0
Number of parameters in AE:  36


/tmp/ipykernel_1061906/351470483.py:84: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0,n_init=10).fit(X_embed)


AE3  0.0
Number of parameters in TAE:  36
epoch 149 loss 0.018861024471892356
TAE1  1.0
Number of parameters in TAE:  84
epoch 149 loss 0.00949009539016212
TAE2  1.0
Number of parameters in TAE:  36
epoch 149 loss 0.17938635345334963
PTAE  1.0


### real data

In [4]:
# kmeans_ari, ae_ari, tae_ari gives the ari of respective algorithms
# hyperparameters: repetition, epochs, lr, reg,  
repetition = 5
results_dict = dict()

csv_file = "results/ae_clust_real.csv"
if not os.path.isfile(csv_file):
    fields = ["data", "rep", "X", "Y", "shape", "noise",
             "kmeans_labels", "kmeans_ari",
             "ae1_num_params", "ae1_lr", "ae1_epochs", "ae1_final_train_loss", "ae1_labels", "ae1_ari", 
             "ae2_num_params", "ae2_lr", "ae2_epochs", "ae2_final_train_loss", "ae2_labels", "ae2_ari", 
             "ae3_num_params", "ae3_lr", "ae3_epochs", "ae3_final_train_loss", "ae3_labels", "ae3_ari",
             "tae1_num_params", "tae1_lr", "tae1_epochs", "tae1_final_train_loss", "tae1_labels", "tae1_ari",
             "tae2_num_params", "tae2_lr", "tae2_epochs", "tae2_final_train_loss", "tae2_labels", "tae2_ari",
             "ptae_num_params", "ptae_lr", "ptae_epochs", "ptae_final_train_loss", "ptae_labels", "ptae_ari", 
             ]
    with open(csv_file, "a") as f:
        writer = csv.writer(f)
        writer.writerow(fields)

lr_l = [0.001, 0.01, 0.1]
epochs = 200
for dataset in ['mnist', 'penguin4', 'iris']:
    if dataset == 'penguin4':
        X,Y,X_noise,n_clusters = real_data_loader(dataset='penguin4', class_labels_list=[0,1,2], normalise_data=True, subsample=False, noise=0)
    elif dataset == 'iris':
        X,Y,X_noise,n_clusters = real_data_loader(dataset='iris', class_labels_list=[0,1,2], normalise_data=False, subsample=False, noise=0)
    elif dataset == 'mnist':
        X,Y,X_noise,n_clusters = real_data_loader(dataset='mnist', class_labels_list=[0,1,2,3,4], num_samples=200, normalise_data=False, subsample=True, noise=0)
    print("shape of dataset: ", X.shape)
    for rep in range(repetition):
        np.random.seed(rep)
        print(rep)
#         X,Y,X_noise,n_clusters = data_generation
        x_idx = torch.tensor(np.arange(X.shape[0]))
        shuffle_idx = torch.randperm(x_idx.shape[0])
        x_idx = x_idx[shuffle_idx]
        X = X[x_idx].float().to(device)
        Y = Y[x_idx].float().to(device)
        
        # k means
        k = KMeans(n_clusters=n_clusters, random_state=rep, n_init=rep+1).fit(X.cpu())
        k_ari = adjusted_rand_score(k.labels_, Y.cpu())
        print('kmeans ', k_ari)
         
        for lr in lr_l:
            res = [dataset, rep, X.cpu().detach().numpy().tolist(), Y.cpu().detach().numpy().tolist(), list(X.shape), 0,
                  k.labels_.tolist(), k_ari]
            
            # Standard one layer AE1 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[1]).to(device)
            ae_l = train_AE(net,X.clone(),Y.clone(), lr=lr,epochs=epochs)
            X_embed = net.get_embedding(X).cpu().detach().numpy()
            kmeans = KMeans(n_clusters=n_clusters, random_state=rep,n_init=rep).fit(X_embed)
            AE_clust_assign = kmeans.labels_
            ari = adjusted_rand_score(AE_clust_assign, Y.cpu())
            ae1_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae1_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])
            res.append(AE_clust_assign.tolist())
            res.append(ari)
            print('AE1 ', ari)

            # Standard two layer AE2 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[2,1]).to(device)
            ae_l = train_AE(net,X.clone(),Y.clone(), lr=lr,epochs=epochs)
            X_embed = net.get_embedding(X).cpu().detach().numpy()
            kmeans = KMeans(n_clusters=n_clusters, random_state=rep,n_init=rep).fit(X_embed)
            AE_clust_assign = kmeans.labels_
            ari = adjusted_rand_score(AE_clust_assign, Y.cpu())
            ae2_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae2_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])
            res.append(AE_clust_assign.tolist())
            res.append(ari)
            print('AE2 ', ari)

            # Standard two layer AE3 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[2,1*n_clusters]).to(device)
            ae_l = train_AE(net,X.clone(),Y.clone(), lr=lr,epochs=epochs)
            X_embed = net.get_embedding(X).cpu().detach().numpy()
            kmeans = KMeans(n_clusters=n_clusters, random_state=rep,n_init=rep).fit(X_embed)
            AE_clust_assign = kmeans.labels_
            ari = adjusted_rand_score(AE_clust_assign, Y.cpu())
            ae3_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae3_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])
            res.append(AE_clust_assign.tolist())
            res.append(ari)
            print('AE3 ', ari)

            # Tensorized AE1 - one layer
            net, tae_l, TAE_clust_assign, _, _, _ = train_TAE(X.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, embed_l=[1], shared=False, epochs=epochs)
            TAE_clust_assign = torch.argmax(TAE_clust_assign,axis=0).cpu().detach().numpy()
            ari = adjusted_rand_score(TAE_clust_assign, Y.cpu())
            tae1_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(tae1_params)
            res.append(lr)
            res.append(epochs)
            res.append(tae_l[-1])
            res.append(TAE_clust_assign.tolist())
            res.append(ari)
            print('TAE1 ', ari)

            # Tensorized AE2 - two layer
            net, tae_l, TAE_clust_assign, _, _, _ = train_TAE(X.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, embed_l=[2,1], shared=False, epochs=epochs)
            TAE_clust_assign = torch.argmax(TAE_clust_assign,axis=0).cpu().detach().numpy()
            ari = adjusted_rand_score(TAE_clust_assign, Y.cpu())
            tae2_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(tae2_params)
            res.append(lr)
            res.append(epochs)
            res.append(tae_l[-1])
            res.append(TAE_clust_assign.tolist())
            res.append(ari)
            print('TAE2 ', ari)

            # Shared TAE
            net, shared_l, SHARED_clust_assign, _, _, _ = train_TAE(X.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, epochs=epochs, embed_l=[2,1], shared=True)
            PTAE_clust_assign = torch.argmax(SHARED_clust_assign,axis=0).cpu().detach().numpy()
            ari = adjusted_rand_score(PTAE_clust_assign, Y.cpu())
            ptae_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ptae_params)
            res.append(lr)
            res.append(epochs)
            res.append(shared_l[-1])
            res.append(PTAE_clust_assign.tolist())
            res.append(ari)
            print('PTAE ', ari)

            with open(csv_file, "a") as f:
                writer = csv.writer(f)
                writer.writerow(res)

MNIST loaded dataset Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
Number of total samples  torch.Size([1000])
shape of dataset:  torch.Size([1000, 784])
4


/dss/dsshome1/lxc0D/apdl004/cluster_specific_representation/cluster_ae/src/data.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X_full[x_idx], dtype=torch.float)
/dss/dsshome1/lxc0D/apdl004/cluster_specific_representation/cluster_ae/src/data.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(Y_full[x_idx])


kmeans  0.7591186456275665
Number of parameters in AE:  1568
AE1  0.12076698526179401
Number of parameters in AE:  3140
AE2  0.11831033074959062
Number of parameters in AE:  3156
AE3  0.34020411357345626
Number of parameters in TAE:  7840
epoch 199 loss 3122.494755859375
TAE1  0.7491182910487174
Number of parameters in TAE:  15700
epoch 199 loss 3082.739341796875
TAE2  0.7451978948793391
Number of parameters in TAE:  3156
epoch 199 loss 3102.813781311035
PTAE  0.7273294581633553
Number of parameters in AE:  1568
AE1  0.08715047243685645
Number of parameters in AE:  3140
AE2  0.11818703868753769
Number of parameters in AE:  3156
AE3  0.3034190544868286
Number of parameters in TAE:  7840
epoch 199 loss 2761.672920959473
TAE1  0.8244280111556831
Number of parameters in TAE:  15700
epoch 199 loss 2686.8864410400392
TAE2  0.7980636861144129
Number of parameters in TAE:  3156
epoch 199 loss 2848.5042033081054
PTAE  0.7199246207376919
Number of parameters in AE:  1568
AE1  0.01749006628801538

## Denoising

### synthetic data

In [4]:
# kmeans_ari, ae_ari, tae_ari gives the ari of respective algorithms
# hyperparameters: repetition, epochs, lr, reg,  
repetition = 5
results_dict = dict()

csv_file = "results/ae_denoising_synthetic.csv"
if not os.path.isfile(csv_file):
    fields = ["data", "rep", "X", "Y", "shape", "noise",
             "ae1_num_params", "ae1_lr", "ae1_epochs", "ae1_final_train_loss",
             "ae2_num_params", "ae2_lr", "ae2_epochs", "ae2_final_train_loss", 
             "ae3_num_params", "ae3_lr", "ae3_epochs", "ae3_final_train_loss", 
             "tae1_num_params", "tae1_lr", "tae1_epochs", "tae1_final_train_loss",
             "tae2_num_params", "tae2_lr", "tae2_epochs", "tae2_final_train_loss", 
             "ptae_num_params", "ptae_lr", "ptae_epochs", "ptae_final_train_loss", 
             ]
    with open(csv_file, "a") as f:
        writer = csv.writer(f)
        writer.writerow(fields)

lr_l = [0.001, 0.01, 0.1]
epochs = 100
noise = 0.1
for (idx, dataset, data_generation) in [
                            (0, 'parallel_line', parallel_line(noise=noise)), 
                            (1, 'orthogonal', orthogonal(noise=noise)), 
                            (2, 'triangle', triangle(noise=noise)), 
                            (3, 'lines_3D', lines_3D(noise=noise))
                             ]:
    for rep in range(repetition):
        np.random.seed(rep)
        print(rep)
        X,Y,X_noise,n_clusters = data_generation
        print("shape of dataset: ", X.shape)
        x_idx = torch.tensor(np.arange(X.shape[0]))
        shuffle_idx = torch.randperm(x_idx.shape[0])
        x_idx = x_idx[shuffle_idx]
        X_noise = X_noise[x_idx].float().to(device)
        X = X[x_idx].float().to(device)
        Y = Y[x_idx].float().to(device)
         
        for lr in lr_l:
            res = [dataset, rep, X.cpu().detach().numpy().tolist(), Y.cpu().detach().numpy().tolist(), list(X.shape), noise,
                  ]
            
            # Standard one layer AE1 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[1]).to(device)
            ae_l = train_AE(net,X_noise.clone(),Y.clone(), lr=lr,epochs=epochs, X_out=X.clone())
            ae1_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae1_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])

            # Standard two layer AE2 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[2,1]).to(device)
            ae_l = train_AE(net,X_noise.clone(),Y.clone(), lr=lr,epochs=epochs, X_out=X.clone())
            ae2_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae2_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])

            # Standard two layer AE3 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[2,1*n_clusters]).to(device)
            ae_l = train_AE(net,X_noise.clone(),Y.clone(), lr=lr,epochs=epochs,X_out=X.clone())
            ae3_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae3_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])

            # Tensorized AE1 - one layer
            net, tae_l, TAE_clust_assign, _, _, _ = train_TAE(X_noise.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, embed_l=[1], shared=False, epochs=epochs, X_out=X.clone())
            tae1_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(tae1_params)
            res.append(lr)
            res.append(epochs)
            res.append(tae_l[-1])

            # Tensorized AE2 - two layer
            net, tae_l, TAE_clust_assign, _, _, _ = train_TAE(X_noise.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, embed_l=[2,1], shared=False, epochs=epochs, X_out=X.clone())
            tae2_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(tae2_params)
            res.append(lr)
            res.append(epochs)
            res.append(tae_l[-1])

            # Shared TAE
            net, shared_l, SHARED_clust_assign, _, _, _ = train_TAE(X_noise.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, epochs=epochs, embed_l=[2,1], shared=True, X_out=X.clone())
            ptae_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ptae_params)
            res.append(lr)
            res.append(epochs)
            res.append(shared_l[-1])

            with open(csv_file, "a") as f:
                writer = csv.writer(f)
                writer.writerow(res)

0
shape of dataset:  torch.Size([150, 5])
Number of parameters in AE:  10
Number of parameters in AE:  24
Number of parameters in AE:  32
Number of parameters in TAE:  30
epoch 99 loss 1.6224375376850366
Number of parameters in TAE:  72
epoch 99 loss 1.4371282520393531
Number of parameters in TAE:  32
epoch 99 loss 1.5511166723320882
Number of parameters in AE:  10
Number of parameters in AE:  24
Number of parameters in AE:  32
Number of parameters in TAE:  30
epoch 99 loss 0.5540691858095428
Number of parameters in TAE:  72
epoch 99 loss 0.22249870121323814
Number of parameters in TAE:  32
epoch 99 loss 0.3822268884566923
Number of parameters in AE:  10
Number of parameters in AE:  24
Number of parameters in AE:  32
Number of parameters in TAE:  30
epoch 99 loss 0.16377197625736395
Number of parameters in TAE:  72
epoch 99 loss 0.12183551267410318
Number of parameters in TAE:  32
epoch 99 loss 0.35368366612742347
1
shape of dataset:  torch.Size([150, 5])
Number of parameters in AE:  1

### real data

In [4]:
# kmeans_ari, ae_ari, tae_ari gives the ari of respective algorithms
# hyperparameters: repetition, epochs, lr, reg,  
repetition = 5
results_dict = dict()

csv_file = "results/ae_denoising_real.csv"
if not os.path.isfile(csv_file):
    fields = ["data", "rep", "X", "Y", "shape", "noise",
             "ae1_num_params", "ae1_lr", "ae1_epochs", "ae1_final_train_loss", 
             "ae2_num_params", "ae2_lr", "ae2_epochs", "ae2_final_train_loss",
             "ae3_num_params", "ae3_lr", "ae3_epochs", "ae3_final_train_loss", 
             "tae1_num_params", "tae1_lr", "tae1_epochs", "tae1_final_train_loss", 
             "tae2_num_params", "tae2_lr", "tae2_epochs", "tae2_final_train_loss",
             "ptae_num_params", "ptae_lr", "ptae_epochs", "ptae_final_train_loss", 
             ]
    with open(csv_file, "a") as f:
        writer = csv.writer(f)
        writer.writerow(fields)

lr_l = [0.001, 0.01, 0.1]
epochs = 200
noise = 0.1
for dataset in ['mnist', 'penguin4', 'iris']:
    if dataset == 'penguin4':
        X,Y,X_noise,n_clusters = real_data_loader(dataset='penguin4', class_labels_list=[0,1,2], normalise_data=True, subsample=False, noise=noise)
    elif dataset == 'iris':
        X,Y,X_noise,n_clusters = real_data_loader(dataset='iris', class_labels_list=[0,1,2], normalise_data=False, subsample=False, noise=noise)
    elif dataset == 'mnist':
        X,Y,X_noise,n_clusters = real_data_loader(dataset='mnist', class_labels_list=[0,1,2,3,4], num_samples=200, normalise_data=False, subsample=True, noise=noise)
    print("shape of dataset: ", X.shape)
    for rep in range(repetition):
        np.random.seed(rep)
        print(rep)
        x_idx = torch.tensor(np.arange(X.shape[0]))
        shuffle_idx = torch.randperm(x_idx.shape[0])
        x_idx = x_idx[shuffle_idx]
        X_noise = X_noise[x_idx].float().to(device)
        X = X[x_idx].float().to(device)
        Y = Y[x_idx].float().to(device)
         
        for lr in lr_l:
            res = [dataset, rep, X.cpu().detach().numpy().tolist(), Y.cpu().detach().numpy().tolist(), list(X.shape), noise,
                  ]
            
            # Standard one layer AE1 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[1]).to(device)
            ae_l = train_AE(net,X_noise.clone(),Y.clone(), lr=lr,epochs=epochs, X_out=X.clone())
            ae1_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae1_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])

            # Standard two layer AE2 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[2,1]).to(device)
            ae_l = train_AE(net,X_noise.clone(),Y.clone(), lr=lr,epochs=epochs, X_out=X.clone())
            ae2_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae2_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])

            # Standard two layer AE3 
            net = Autoencoder(in_feature=X.shape[1],embed_l=[2,1*n_clusters]).to(device)
            ae_l = train_AE(net,X_noise.clone(),Y.clone(), lr=lr,epochs=epochs,X_out=X.clone())
            ae3_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ae3_params)
            res.append(lr)
            res.append(epochs)
            res.append(ae_l[-1])

            # Tensorized AE1 - one layer
            net, tae_l, TAE_clust_assign, _, _, _ = train_TAE(X_noise.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, embed_l=[1], shared=False, epochs=epochs, X_out=X.clone())
            tae1_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(tae1_params)
            res.append(lr)
            res.append(epochs)
            res.append(tae_l[-1])

            # Tensorized AE2 - two layer
            net, tae_l, TAE_clust_assign, _, _, _ = train_TAE(X_noise.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, embed_l=[2,1], shared=False, epochs=epochs, X_out=X.clone())
            tae2_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(tae2_params)
            res.append(lr)
            res.append(epochs)
            res.append(tae_l[-1])

            # Shared TAE
            net, shared_l, SHARED_clust_assign, _, _, _ = train_TAE(X_noise.clone(),Y.clone(), n_clusters=n_clusters, lr=lr, reg=0.1, epochs=epochs, embed_l=[2,1], shared=True, X_out=X.clone())
            ptae_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
            res.append(ptae_params)
            res.append(lr)
            res.append(epochs)
            res.append(shared_l[-1])

            with open(csv_file, "a") as f:
                writer = csv.writer(f)
                writer.writerow(res)

/dss/dsshome1/lxc0D/apdl004/cluster_specific_representation/cluster_ae/src/data.py:102: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  peng = peng.replace({'species': peng_dict})


shape of dataset:  torch.Size([334, 4])
4
Number of parameters in AE:  8
Number of parameters in AE:  20
Number of parameters in AE:  28
Number of parameters in TAE:  24
epoch 199 loss 0.0027628246855397128
Number of parameters in TAE:  60
epoch 199 loss 0.002336736495613921
Number of parameters in TAE:  28
epoch 199 loss 0.002442312783038475
Number of parameters in AE:  8
Number of parameters in AE:  20
Number of parameters in AE:  28
Number of parameters in TAE:  24
epoch 199 loss 0.0019748119421182674
Number of parameters in TAE:  60
epoch 199 loss 0.001893412104921421
Number of parameters in TAE:  28
epoch 199 loss 0.0018994574654466769
Number of parameters in AE:  8
Number of parameters in AE:  20
Number of parameters in AE:  28
Number of parameters in TAE:  24
epoch 199 loss 0.0018962889833804178
Number of parameters in TAE:  60
epoch 199 loss 0.001892274151119462
Number of parameters in TAE:  28
epoch 199 loss 0.001885063430144257
